In [2]:
# Load LSTM network and generate text
import sys
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.utils import np_utils
import h5py
import re
import tensorflow as tf
import math
import pandas as pd
from pprint import pprint

Using TensorFlow backend.


In [23]:
bc_df=pd.read_csv("/home/kavin/Silo/College Work/AI/DataSets/brown_tag_lines.txt",sep="_", names=("word","pos_tag"))

In [18]:
bc_df.head()

,word,pos_tag
0,the,at
1,fulton,np
2,county,nn
3,grand,jj
4,jury,nn


In [24]:
words=[]
for word in bc_df.itertuples():
    words.append(word[1].lower())
uniques_words=pd.Series(words).unique())

SyntaxError: invalid syntax (<ipython-input-24-b4eb36d05c50>, line 4)

In [25]:
unique_words=pd.Series(bc_df["word"]).unique()
n_vocab=len(unique_words)

In [26]:
bag_of_words={}
index_to_words={}
index=0
for word in unique_words:
    bag_of_words[word]=index
    index_to_words[index]=word
    index+=1

In [27]:
unique_tags=bc_df["pos_tag"].unique()
bag_of_tags={}
index_to_tags={}
index=0
for tag in unique_tags:
    bag_of_tags[tag]=index
    index_to_tags[index]=tag
    index+=1

In [43]:
bag_of_tags

{nan: 312,
 '(': 61,
 ')': 62,
 '*': 50,
 '*-nc': 231,
 ',': 13,
 ',-nc': 213,
 '--': 56,
 '.': 11,
 '.-nc': 215,
 '.|sb01:1': 130,
 '.|sc01:1': 139,
 '.|sd01:1': 149,
 '.|se01:1': 152,
 '.|sf01:1': 161,
 '.|sg01:1': 179,
 '.|sh01:1': 202,
 '.|sj01:1': 204,
 '.|sk01:1': 262,
 '.|sl01:1': 280,
 '.|sm01:1': 287,
 '.|sn01:1': 288,
 '.|sp01:1': 307,
 '.|sr01:1': 311,
 ':': 52,
 'abl': 74,
 'abn': 44,
 'abn-nc': 248,
 'abx': 35,
 'ap': 27,
 'ap$': 104,
 'ap+ap-nc': 257,
 'ap-nc': 234,
 'at': 0,
 'at-nc': 172,
 'be': 39,
 'bed': 57,
 'bed*': 114,
 'bed-nc': 236,
 'bedz': 18,
 'bedz*': 66,
 'bedz-nc': 225,
 'beg': 59,
 'bem': 83,
 'bem*': 275,
 'bem-nc': 253,
 'ben': 20,
 'ber': 28,
 'ber*': 112,
 'ber*-nc': 233,
 'ber-nc': 208,
 'bez': 41,
 'bez*': 103,
 'bez-nc': 245,
 'cc': 17,
 'cc-nc': 237,
 'cd': 37,
 'cd$': 111,
 'cd-nc': 218,
 'cs': 8,
 'cs-nc': 241,
 'do': 48,
 'do*': 89,
 'do+ppss': 299,
 'do-nc': 235,
 'dod': 26,
 'dod*': 65,
 'dod-nc': 251,
 'doz': 63,
 'doz*': 90,
 'dt': 24,
 'dt

In [22]:
pos_index=[]
word_index=[]
for word in bc_df.itertuples():
    word_index.append(bag_of_words[word[1]])
    pos_index.append(bag_of_tags[word[2]])
bc_df["word_index"]=word_index
bc_df["pos_tag_index"]=pos_index

In [23]:
split_index=int(math.floor(len(bc_df)*0.75))
split_index

853100

In [24]:
train, test = bc_df[:split_index], bc_df[split_index:]

In [28]:
train.head()
train["pos_tag_index"][1]

1

In [30]:
seq_length = 10
dataX = []
dataY = []
input_line=[]
output_line=[]
for i in range(0, 10000, 1):
    input_line=train["pos_tag_index"][i:i+seq_length]
    output_line=train["pos_tag_index"][i+seq_length]
    dataX.append(input_line)
    dataY.append(output_line)
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  10000


In [20]:
len(dataY)

990

In [38]:
import json
with open('pos_bag_of_tags.json', 'w') as fp:
    json.dump(bag_of_tags, fp)
with open('pos_bag_of_words.json', 'w') as fp:
    json.dump(bag_of_words, fp)
with open('pos_index_to_tags.json', 'w') as fp:
    json.dump(bag_of_tags, fp)
with open('pos_index_to_words.json', 'w') as fp:
    json.dump(bag_of_words, fp)

In [33]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (len(dataX), 10, 1))
    # normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [55]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# define the checkpoint
filepath="Donna Files/tagger_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=1000, batch_size=128, callbacks=callbacks_list)

Epoch 1/1000
10000/10000 [==============================] - 3s - loss: 3.4152     
Epoch 2/1000
10000/10000 [==============================] - 2s - loss: 3.1757     
Epoch 3/1000
10000/10000 [==============================] - 2s - loss: 3.1667     
Epoch 4/1000
10000/10000 [==============================] - 2s - loss: 3.1686     
Epoch 5/1000
10000/10000 [==============================] - 2s - loss: 3.1692     
Epoch 6/1000
10000/10000 [==============================] - 2s - loss: 3.1608     
Epoch 7/1000
10000/10000 [==============================] - 2s - loss: 3.1632     
Epoch 8/1000
10000/10000 [==============================] - 2s - loss: 3.1601     
Epoch 9/1000
10000/10000 [==============================] - 2s - loss: 3.1603     
Epoch 10/1000
10000/10000 [==============================] - 2s - loss: 3.1593     
Epoch 11/1000
10000/10000 [==============================] - 2s - loss: 3.1562     
Epoch 12/1000
10000/10000 [==============================] - 2s - loss: 3.1601     
E

10000/10000 [==============================] - 2s - loss: 3.1488     
Epoch 79/1000
10000/10000 [==============================] - 2s - loss: 3.1474     
Epoch 80/1000
10000/10000 [==============================] - 2s - loss: 3.1494     
Epoch 81/1000
10000/10000 [==============================] - 2s - loss: 3.1484     
Epoch 82/1000
10000/10000 [==============================] - 2s - loss: 3.1493     
Epoch 83/1000
10000/10000 [==============================] - 2s - loss: 3.1477     
Epoch 84/1000
10000/10000 [==============================] - 2s - loss: 3.1479     
Epoch 85/1000
10000/10000 [==============================] - 2s - loss: 3.1485     
Epoch 86/1000
10000/10000 [==============================] - 2s - loss: 3.1487     
Epoch 87/1000
10000/10000 [==============================] - 2s - loss: 3.1477     
Epoch 88/1000
10000/10000 [==============================] - 2s - loss: 3.1467     
Epoch 89/1000
10000/10000 [==============================] - 2s - loss: 3.1476     
Epoch 

10000/10000 [==============================] - 2s - loss: 3.1471     
Epoch 162/1000
10000/10000 [==============================] - 2s - loss: 3.1460     
Epoch 163/1000
10000/10000 [==============================] - 2s - loss: 3.1453     
Epoch 164/1000
10000/10000 [==============================] - 2s - loss: 3.1455     
Epoch 165/1000
10000/10000 [==============================] - 2s - loss: 3.1461     
Epoch 166/1000
10000/10000 [==============================] - 2s - loss: 3.1462     
Epoch 167/1000
10000/10000 [==============================] - 2s - loss: 3.1472     
Epoch 168/1000
10000/10000 [==============================] - 2s - loss: 3.1468     
Epoch 169/1000
10000/10000 [==============================] - 2s - loss: 3.1462     
Epoch 170/1000
10000/10000 [==============================] - 2s - loss: 3.1467     
Epoch 171/1000
10000/10000 [==============================] - 2s - loss: 3.1465     
Epoch 172/1000
10000/10000 [==============================] - 2s - loss: 3.1467 

10000/10000 [==============================] - 2s - loss: 3.1450     
Epoch 247/1000
10000/10000 [==============================] - 2s - loss: 3.1442     
Epoch 248/1000
10000/10000 [==============================] - 2s - loss: 3.1450     
Epoch 249/1000
10000/10000 [==============================] - 2s - loss: 3.1452     
Epoch 250/1000
10000/10000 [==============================] - 2s - loss: 3.1450     
Epoch 251/1000
10000/10000 [==============================] - 2s - loss: 3.1446     
Epoch 252/1000
10000/10000 [==============================] - 2s - loss: 3.1444     
Epoch 253/1000
10000/10000 [==============================] - 2s - loss: 3.1446     
Epoch 254/1000
10000/10000 [==============================] - 2s - loss: 3.1448     
Epoch 255/1000
10000/10000 [==============================] - 2s - loss: 3.1446     
Epoch 256/1000
10000/10000 [==============================] - 2s - loss: 3.1456     
Epoch 257/1000
10000/10000 [==============================] - 2s - loss: 3.1434 

10000/10000 [==============================] - 2s - loss: 3.1408     
Epoch 329/1000
10000/10000 [==============================] - 2s - loss: 3.1424     
Epoch 330/1000
10000/10000 [==============================] - 2s - loss: 3.1409     
Epoch 331/1000
10000/10000 [==============================] - 2s - loss: 3.1423     
Epoch 332/1000
10000/10000 [==============================] - 2s - loss: 3.1403     
Epoch 333/1000
10000/10000 [==============================] - 2s - loss: 3.1403     
Epoch 334/1000
10000/10000 [==============================] - 2s - loss: 3.1416     
Epoch 335/1000
10000/10000 [==============================] - 2s - loss: 3.1410     
Epoch 336/1000
10000/10000 [==============================] - 2s - loss: 3.1408     
Epoch 337/1000
10000/10000 [==============================] - 2s - loss: 3.1393     
Epoch 338/1000
10000/10000 [==============================] - 2s - loss: 3.1393     
Epoch 339/1000
10000/10000 [==============================] - 2s - loss: 3.1389 

10000/10000 [==============================] - 2s - loss: 3.1289     
Epoch 405/1000
10000/10000 [==============================] - 2s - loss: 3.1279     
Epoch 406/1000
10000/10000 [==============================] - 2s - loss: 3.1276     
Epoch 407/1000
10000/10000 [==============================] - 2s - loss: 3.1281     
Epoch 408/1000
10000/10000 [==============================] - 2s - loss: 3.1261     
Epoch 409/1000
10000/10000 [==============================] - 2s - loss: 3.1264     
Epoch 410/1000
10000/10000 [==============================] - 2s - loss: 3.1269     
Epoch 411/1000
10000/10000 [==============================] - 2s - loss: 3.1242     
Epoch 412/1000
10000/10000 [==============================] - 2s - loss: 3.1245     
Epoch 413/1000
10000/10000 [==============================] - 2s - loss: 3.1247     
Epoch 414/1000
10000/10000 [==============================] - 2s - loss: 3.1245     
Epoch 415/1000
10000/10000 [==============================] - 2s - loss: 3.1231 

10000/10000 [==============================] - 2s - loss: 3.0875     
Epoch 476/1000
10000/10000 [==============================] - 2s - loss: 3.0872     
Epoch 477/1000
10000/10000 [==============================] - 2s - loss: 3.0868     
Epoch 478/1000
10000/10000 [==============================] - 2s - loss: 3.0866     
Epoch 479/1000
10000/10000 [==============================] - 2s - loss: 3.0850     
Epoch 480/1000
10000/10000 [==============================] - 2s - loss: 3.0825     
Epoch 481/1000
10000/10000 [==============================] - 2s - loss: 3.0826     
Epoch 482/1000
10000/10000 [==============================] - 2s - loss: 3.0835     
Epoch 483/1000
10000/10000 [==============================] - 2s - loss: 3.0831     
Epoch 484/1000
10000/10000 [==============================] - 2s - loss: 3.0828     
Epoch 485/1000
10000/10000 [==============================] - 2s - loss: 3.0817     
Epoch 486/1000
10000/10000 [==============================] - 2s - loss: 3.0794 

10000/10000 [==============================] - 2s - loss: 3.0257     
Epoch 545/1000
10000/10000 [==============================] - 2s - loss: 3.0279     
Epoch 546/1000
10000/10000 [==============================] - 2s - loss: 3.0169     
Epoch 547/1000
10000/10000 [==============================] - 2s - loss: 3.0214     
Epoch 548/1000
10000/10000 [==============================] - 2s - loss: 3.0209     
Epoch 549/1000
10000/10000 [==============================] - 2s - loss: 3.0192     
Epoch 550/1000
10000/10000 [==============================] - 2s - loss: 3.0168     
Epoch 551/1000
10000/10000 [==============================] - 2s - loss: 3.0172     
Epoch 552/1000
10000/10000 [==============================] - 2s - loss: 3.0135     
Epoch 553/1000
10000/10000 [==============================] - 2s - loss: 3.0113     
Epoch 554/1000
10000/10000 [==============================] - 2s - loss: 3.0148     
Epoch 555/1000
10000/10000 [==============================] - 2s - loss: 3.0106 

10000/10000 [==============================] - 2s - loss: 2.9644     
Epoch 582/1000
10000/10000 [==============================] - 2s - loss: 2.9594     
Epoch 583/1000
10000/10000 [==============================] - 2s - loss: 2.9633     
Epoch 584/1000
10000/10000 [==============================] - 2s - loss: 2.9515     
Epoch 585/1000
10000/10000 [==============================] - 2s - loss: 2.9562     
Epoch 586/1000
10000/10000 [==============================] - 2s - loss: 2.9557     
Epoch 587/1000
10000/10000 [==============================] - 2s - loss: 2.9469     
Epoch 588/1000
10000/10000 [==============================] - 2s - loss: 2.9474     
Epoch 589/1000
10000/10000 [==============================] - 2s - loss: 2.9477     
Epoch 590/1000
10000/10000 [==============================] - 2s - loss: 2.9445     
Epoch 591/1000
10000/10000 [==============================] - 2s - loss: 2.9489     
Epoch 592/1000
10000/10000 [==============================] - 2s - loss: 2.9367 

10000/10000 [==============================] - 2s - loss: 2.6539     
Epoch 648/1000
10000/10000 [==============================] - 2s - loss: 2.6277     
Epoch 649/1000
10000/10000 [==============================] - 2s - loss: 2.6395     
Epoch 650/1000
10000/10000 [==============================] - 2s - loss: 2.6129     
Epoch 651/1000
10000/10000 [==============================] - 2s - loss: 2.6098     
Epoch 652/1000
10000/10000 [==============================] - 2s - loss: 2.6169     
Epoch 653/1000
10000/10000 [==============================] - 2s - loss: 2.5993     
Epoch 654/1000
10000/10000 [==============================] - 2s - loss: 2.5815     
Epoch 655/1000
10000/10000 [==============================] - 2s - loss: 2.5722     
Epoch 656/1000
10000/10000 [==============================] - 2s - loss: 2.5744     
Epoch 657/1000
10000/10000 [==============================] - 2s - loss: 2.5654     
Epoch 658/1000
10000/10000 [==============================] - 2s - loss: 2.5547 

In [37]:
model.save("Donna Files/pos_tagger_model.h5")

In [48]:
# define the checkpoint
seq_length = 10
dataX = []
dataY = []
input_line=[]
output_line=[]
for i in range(10000, 20000, 1):
    input_line=train["pos_tag_index"][i:i+seq_length]
    output_line=train["pos_tag_index"][i+seq_length]
    dataX.append(input_line)
    dataY.append(output_line)
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (len(dataX), 10, 1))
    # normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Patterns:  10000


In [36]:
model=load_model("Donna Files/pos_tagger_model.h5")

filepath="Donna Files/tagger_weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=1000, batch_size=128, callbacks=callbacks_list)

NameError: name 'X' is not defined

In [37]:
model=load_model("Donna Files/pos_tagger_model.h5")

# load the network weights
filename = "Donna Files/tagger_weights-improvement-997-0.8726.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [29]:
# pick a random seed
input_line="Why on earth would you build something like that"
pattern = input_line.split()
print "Seed:"
print "\"", ' '.join([str(value) for value in pattern]), "\""
indexed_input=[]
([indexed_input.append(bag_of_words[value.lower()]) for value in pattern])


Seed:
" Why on earth would you build something like that "


[None, None, None, None, None, None, None, None, None]

In [30]:
pattern=indexed_input

In [31]:
seq_length=10

In [6]:
pattern=[]

In [32]:
pattern+=list(np.zeros(seq_length-len(pattern)))

In [33]:
np.zeros(seq_length-len(pattern))

array([], dtype=float64)

In [35]:
# generate characters
for i in range(10):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(303)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = index_to_tags[index]
	#seq_in = [index_to_words[value] for value in pattern]
	sys.stdout.write(result+" ")
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print "\nDone."

md* ( pps+hvz in dt$ fw-nn fw-nn md* dt$ ( 
Done.
